In [ ]:
!pip3 install transformers
!pip3 install torch torchvision
!pip3 install datasets
!pip3 install tqdm
!pip3 install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, DataCollatorWithPadding
from tqdm import tqdm, trange
import torch.nn.functional as F
import datasets
from datasets import load_dataset, concatenate_datasets

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


#raw_datasets = load_dataset("pile-of-law/pile-of-law", 'us_bills')
#law_list = ['acus_reports', 'cc_casebooks']

#dataset = [load_dataset('pile-of-law/pile-of-law', law, split= 'train') for law in law_list]

#n = len(dataset)

def tokenize_function(example):
    return tokenizer(example["text"], max_length=512, padding='max_length', truncation=True, return_tensors='pt')

law_list = ['acus_reports', 'cc_casebooks', 'cfpb_creditcard_contracts',
            'ed_policy_guidance', 'eoir', 
            'frcp', 'fre', 'ftc_advisory_opinions', 'hhs_alj_opinions', 
            'uscode']
law_list1 = ['acus_reports', 'cc_casebooks', 'cfpb_creditcard_contracts',
            'ed_policy_guidance', 'eoir', 
            'frcp', 'fre', 'ftc_advisory_opinions', 'hhs_alj_opinions', 
            'uscode', "cfr", "scotus_oral_arguments", "state_codes"]

dataset = [load_dataset('pile-of-law/pile-of-law', law, split= 'train') for law in law_list1]

n = len(dataset)
def get_mini_dataset(dataset, subset_size=1):
    subset = [None] * n
    # validation_subset = [] 
    t = []
#     start = time.time()
    for i in range(n):        
        # Append a new subset dataset for the `train` set
        subset[i] = dataset[i].select(range(dataset[i].num_rows//subset_size))
#         t.append(time.time()-start)
#         start = time.time()

    # Create a new DatasetDict with the subset datasets
    a = [i.num_rows//subset_size for i in dataset]
    print(a)
#     print(t)
    merged_dataset = concatenate_datasets(subset)
    subset_dataset = datasets.DatasetDict({'train': merged_dataset})#, 'validation': validation_subset})
    return subset_dataset

mini_raw_datasets = get_mini_dataset(dataset, subset_size=1)

INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/acus_reports/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/acus_reports/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingf

[181, 59, 2023, 507, 1671, 69, 51, 112, 2083, 43, 182, 5997, 157]


In [ ]:

# encoded_input = tokenizer(raw_datasets['train'][3]['text'], return_tensors='pt')
batch = mini_raw_datasets.map(tokenize_function, batched = True)
batch = batch.remove_columns(mini_raw_datasets["train"].column_names)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = 'max_length', max_length = 10)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='m')
batch.set_format("torch")
#special_tokens = tokenizer.special_tokens_map
#special_tokens
#batch['input_ids']

In [ ]:
print(len(mini_raw_datasets["train"]))


13135


In [ ]:
special_tokens = tokenizer.special_tokens_map
special_tokens

{'bos_token': '<startofstring>',
 'eos_token': '<endofstring>',
 'unk_token': '<|endoftext|>',
 'pad_token': '<pad>'}

In [ ]:
input_ids = torch.stack([x['input_ids'] for x in batch['train']])
mask = torch.stack([x['attention_mask'] for x in batch['train']])
#token_type_ids = torch.stack([x['token_type_ids'] for x in batch['train']])

encodings = {'input_ids': input_ids, 'attention_mask': mask}

class GPT2LawData(Dataset):
    def __init__(self, encodings):
        self.X_encoded = encodings

    def __len__(self):
        return self.X_encoded['input_ids'].shape[0]

    def __getitem__(self, idx):
        return (self.X_encoded['input_ids'][idx],  self.X_encoded['attention_mask'][idx]) 

In [ ]:
dataset = GPT2LawData(encodings)
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
input_ids.shape
model.transformer.wte.weight.shape[0] == len(tokenizer)

True

In [ ]:
print(torch.unique(input_ids))

tensor([    0,     1,     2,  ..., 50252, 50254, 50257])


In [ ]:
print(train_dataloader.batch_sampler.batch_size)
print(len(train_dataloader))

16
821


In [ ]:
vocab_size = model.config.vocab_size
print("Vocabulary size:", vocab_size)

Vocabulary size: 50261


In [ ]:
def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a)
    output = tokenizer.decode(output[0])
    return output

In [ ]:
# model = GPT2LMHeadModel.from_pretrained("gpt2")
optim = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# activate training mode
model.train()
# initialize optimizer

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch[0].to(device)
        #input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        #attention_mask = batch[1].to(device)
        #labels = batch['labels'].to(device)
       # token_type_ids = batch['token_type_ids'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask, labels = input_ids)
        # extract loss
        loss, _ = outputs[:2]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    #print(infer("hello how are you"))

Epoch 0:  24%|██▍       | 199/821 [1:03:52<2:49:32, 16.35s/it, loss=3.27]

In [ ]:
# Specify the file path

import torch


drive_path = '/content/gdrive/My Drive/saved_model/'

file_path = 'model_GPT2_full_new_more_data.pth'

# Save the model
torch.save(model.state_dict(), drive_path + file_path)

In [ ]:
from torch import nn
input_dim = 10
embedding_dim = 2
embedding = nn.Embedding(input_dim, embedding_dim)
err = True
if err:
    #Any input more than input_dim - 1, here input_dim = 10
    #Any input less than zero
    input_to_embed = torch.tensor([10])
else:
    input_to_embed = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
embed = embedding(input_to_embed)
print(embed)